# Assignment

In this assignment we want to get comfortable with loading and manipulating data in Python. While future assignments will focus more using structured data which we can load into a `DataFrame` using `pandas`, this assignment is focused on semi-structured data and how we can "flatten" it and then load it into other formats. The objective is to see how data flows in Python from one object to another and what advantages and disadvantages each offers.

Let's read the `books.json` data set and display the first item in it.

In [1]:
import json
with open('../data/books.json', encoding = 'utf-8') as f:
    books_dict = json.load(f)

from pprint import pprint
pprint(books_dict[0]) # print information for the first book

{'_id': 1,
 'authors': ['W. Frank Ableson', 'Charlie Collins', 'Robi Sen'],
 'categories': ['Open Source', 'Mobile'],
 'isbn': '1933988673',
 'longDescription': 'Android is an open source mobile phone platform based on '
                    'the Linux operating system and developed by the Open '
                    'Handset Alliance, a consortium of over 30 hardware, '
                    'software and telecom companies that focus on open '
                    'standards for mobile devices. Led by search giant, '
                    'Google, Android is designed to deliver a better and more '
                    'open and cost effective mobile experience.    Unlocking '
                    "Android: A Developer's Guide provides concise, hands-on "
                    'instruction for the Android operating system and '
                    'development tools. This book teaches important '
                    'architectural concepts in a straightforward writing style '
                    

1. Write a program that goes through the entire data and extracts the following information:  <span style="color:red" float:right>[4 point]</span>

  - title of the book
  - name of the first author
  - name of the second author (if book has more than one author)
  - number of authors
  - ISBN
  - if the word "data" is in the book's description
  - the number of words in the book's description
  - the year the book was published

  Of course because JSON data doesn't necessarily enforce any sort of schema, we can't be sure that the information we are trying to extract exists for every book. For example, if the book only has one author, then there is no second author. So use `try` and `except` as you loop through every book and skip to the next item every time some information is missing.

  Store the extracted data in a list named `rows` whose elements are tuples, one tuple per book. For example, the first element of `rows` stores the tuple for the first book and should look like this:

        ('Unlocking Android', 'W. Frank Ableson', 'Charlie Collins', 3, '1933988673', True, 252, 2009)

## Problem Approach

We will use a for loop to capture 8 attributes of each book in books_dict. However since not all attributes may exist for each book, we use try and except to loop through everybook and assign none to non existent attributes. We will also store them in a list called rows.

In [2]:
from datetime import datetime # we will use this to extract years in publishedDate later

# create a list to store the tuples
rows = []

# use a for loop to store one tuple per book in rows
for book in books_dict:

    # check for first author
    try:
        author1 = book['authors'][0]
    except:
        author1 = None
    
    # check for second author
    try:
        author2 = book['authors'][1]
    except:
        author2 = None
    
    #check data and word count in short description
    try:
        data_inshort = "data" in book['shortDescription']
        words_inshort = len(book['shortDescription'].split())
    except:
        data_inshort = False
        words_inshort = 0

    #check for isbn
    try:
        isbn = book['isbn']
    except:
        isbn = None

    # check data and word count in long description
    try:
        data_inlong = "data" in book['longDescription']
        words_inlong = len(book['longDescription'].split())
    except:
        data_inlong = False
        words_inlong = 0

    # check if data exists in either short or long description
    data = any([data_inshort, data_inlong])

    # check for year in publishedDate
    try:
        year = datetime.fromisoformat(book['publishedDate']['$date'][:19]).year
    except:
        year = None
    
    row = (
        book['title'], # book title
        author1, # first author
        author2, # second author
        len(book['authors']), # number of authors
        isbn, # ISBN
        data, # check if "data" is in book's description
        words_inshort + words_inlong, # total number of words in the books' description
        year # year published
    )   
    rows.append(row)



Now we want to check if the codes worked by checking how many rows have been stored and print a row to see the values of the 8 attributes.

In [3]:
# check how many rows in this list
print (len(rows))

431


In [4]:
# check the values of a random row
print(rows[20])

('Hibernate in Action', 'Christian Bauer', 'Gavin King', 2, '193239415X', True, 199, 2004)


2. Save the content of `rows` in a SQL-like table using `sqlite3`, and choose the appropriate column types. <span style="color:red" float:right>[2 point]</span> 

  As your column names use the following:

  - `title`
  - `author_1`
  - `author_2`
  - `num_authors`
  - `isbn`
  - `has_data`
  - `desc_len`
  - `year_published`

In [5]:
import sqlite3

# create sqlite3 connection
connection = sqlite3.connect(':memory:') 
cursor = connection.cursor()

cursor.execute('''CREATE TABLE books_all
             (title text, author1 text, author2 text, num_of_authors integer, isbn text, has_data_in_description boolean, description_length integer, years_published integer)''')

cursor.executemany('INSERT INTO books_all VALUES (?,?,?,?,?,?,?,?)', rows)

connection.commit() # save the changes

In [6]:
# use below codes to check if this worked. I have commented the codes after checking.
# books_table = cursor.execute("SELECT * FROM books_all").fetchall()
# books_table

3. Write a SQL query against the table to show all books that (1) contain the word "data" and (2) have more than 3 authors. Store the result of the query in an object called `books_table`, then close the connection. <span style="color:red" float:right>[2 point]</span>

In [7]:
import requests
import json

# Select the books that contain word "data" and have more than 3 authors

books_table = cursor.execute("SELECT * FROM books_all WHERE has_data_in_description = 1 AND num_of_authors >3").fetchall()
books_table

[('Flex 4 in Action',
  'Tariq Ahmed',
  'Dan Orlando',
  4,
  '1935182420',
  1,
  329,
  2010),
 ('Spring Dynamic Modules in Action',
  'Arnaud Cogoluegnes',
  'Thierry Templier',
  4,
  '1935182307',
  1,
  289,
  2010),
 ('Android in Practice',
  'Charlie Collins',
  'Michael D. Galpin',
  4,
  '1935182927',
  1,
  277,
  2011),
 ('Portlets and Apache Portals',
  'Stefan Hepper',
  'Peter Fischer',
  5,
  None,
  1,
  174,
  2005),
 ('Struts in Action',
  'Ted N. Husted',
  'Cedric Dumoulin',
  4,
  '1932394249',
  1,
  137,
  2002),
 ('Taming Text',
  'Grant S. Ingersoll',
  'Thomas S. Morton',
  4,
  '193398838X',
  1,
  199,
  2012),
 ('Groovy in Action',
  'Dierk Koenig with Andrew Glover',
  'Paul King',
  4,
  '1932394842',
  1,
  235,
  2007),
 ('Groovy in Action, Second Edition',
  'Dierk König',
  'Guillaume Laforge',
  8,
  '1935182447',
  1,
  336,
  None),
 ('NHibernate in Action',
  'Pierre Henri Kuate',
  'Tobin Harris',
  5,
  '1932394923',
  1,
  222,
  2009),
 ('Tu

SQL tables are not the only way, and definitely not the most straightforward way to store and manipulate data in Python. A format that's more popular with data scientists is to use the `pandas` library to create a `DataFrame`. This library has a lot of functionality that makes it easy to run the common tasks data scientists do with data.

4. Read the data from the above query into a `DataFrame` and call it `books_df`. HINT: Use `pd.DataFrame` and specify the names to use for the columns. <span style="color:red" float:right>[1 point]</span>

In [8]:
import pandas as pd
import time

# read the queried data into a DataFrame
books_df = pd.DataFrame(
    books_table,
    columns=['title', 'author1', 'author2', 'num_of_authors', 'isbn', 'has_data_in_description', 'description_length', 'years_published']
)


5. Display the first few columns of a `DataFrame` by calling its `head` method. <span style="color:red" float:right>[1 point]</span>

In [9]:
# use head to display first few columns
books_df.head()

,title,author1,author2,num_of_authors,isbn,has_data_in_description,description_length,years_published
0,Flex 4 in Action,Tariq Ahmed,Dan Orlando,4,1935182420,1,329,2010.0
1,Spring Dynamic Modules in Action,Arnaud Cogoluegnes,Thierry Templier,4,1935182307,1,289,2010.0
2,Android in Practice,Charlie Collins,Michael D. Galpin,4,1935182927,1,277,2011.0
3,Portlets and Apache Portals,Stefan Hepper,Peter Fischer,5,None,1,174,2005.0
4,Struts in Action,Ted N. Husted,Cedric Dumoulin,4,1932394249,1,137,2002.0


Remember how earlier we said that a `DataFrame` is built on top of `numpy` arrays? Another way of saying it is that a `DataFrame` is an **abstraction** on top of `numpy` arrays: i.e. a `DataFrame` is a more **high-level** object than a `numpy` array. 

6. Call the `values` attribute of your `DataFrame` to convert it into a numpy array and display the first 3 elements of the array. <span style="color:red" float:right>[1 point]</span>

In [10]:
#  call the values of dataframe and convert them into a NumPy array
books_array = books_df.values

# display the first 3 elements (rows) of the array
books_array[:3]

array([['Flex 4 in Action', 'Tariq Ahmed', 'Dan Orlando', 4,
        '1935182420', 1, 329, 2010.0],
       ['Spring Dynamic Modules in Action', 'Arnaud Cogoluegnes',
        'Thierry Templier', 4, '1935182307', 1, 289, 2010.0],
       ['Android in Practice', 'Charlie Collins', 'Michael D. Galpin', 4,
        '1935182927', 1, 277, 2011.0]], dtype=object)

Now you can judge which object is more "user-friendly". That's one of the things that abstractions allow us to do: build more user-friendly (abstract) objects from less user-friendly (but more fundamental) objects.

# End of assignment

## Conclusion
A pandas dataframe looks more "user-friendly" to read because it clearly shows rows and columns. While the array has all the information, it is harder to read because of the different lengths of the text in each attribute.   